In [1]:
import os

In [2]:
%pwd

'c:\\Project\\Insurance-ml-Project\\research'

In [3]:
os.chdir("../")


In [4]:
%pwd

'c:\\Project\\Insurance-ml-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [6]:

from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [8]:
import os
from src.mlProject.logging import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import os

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def train_test_spliting(self):
        data = pd.read_csv(self.config.data_path)

        data.dropna(inplace=True)

        categorical_cols = ['sex', 'smoker', 'region']
        data_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

        X = data_encoded.drop('expenses', axis=1)
        y = data_encoded['expenses']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

        train_data = pd.concat([X_train, y_train], axis=1)
        test_data = pd.concat([X_test, y_test], axis=1)
        
        train_data.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test_data.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info("Splitted data into training and test sets")
        logger.info(f"Training data shape: {train_data.shape}")
        logger.info(f"Test data shape: {test_data.shape}")

        print("Training data shape:", train_data.shape)
        print("Test data shape:", test_data.shape)

In [10]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    print(e) 

[2024-03-13 18:35:48,812: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-13 18:35:48,839: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-13 18:35:48,848: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-13 18:35:48,852: INFO: common: created directory at: artifacts]
[2024-03-13 18:35:48,855: INFO: common: created directory at: artifacts/data_transformation]


[2024-03-13 18:35:48,945: INFO: 3864523879: Splitted data into training and test sets]
[2024-03-13 18:35:48,945: INFO: 3864523879: Training data shape: (1003, 9)]
[2024-03-13 18:35:48,945: INFO: 3864523879: Test data shape: (335, 9)]
Training data shape: (1003, 9)
Test data shape: (335, 9)


In [21]:
train_data = pd.read_csv("artifacts/data_transformation/train.csv")
print(train_data.head())  # Print the first few rows of the dataframe
print(train_data.info())  # Print the summary information about the dataframe
print(train_data.describe())  # Print the summary statistics of numerical columns in the dataframe
print(train_data.isnull().sum())  # Print the number of missing values in each column


   age     bmi  children  sex_male  smoker_yes  region_northwest  \
0   24  23.655         0      True       False              True   
1   28  26.510         2     False       False             False   
2   51  39.700         1      True       False             False   
3   47  36.080         1      True        True             False   
4   46  28.900         2     False       False             False   

   region_southeast  region_southwest     expenses  
0             False             False   2352.96845  
1              True             False   4340.44090  
2             False              True   9391.34600  
3              True             False  42211.13820  
4             False              True   8823.27900  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1003 entries, 0 to 1002
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age               1003 non-null   int64  
 1   bmi               100